In [289]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

df = pd.read_csv("../input/flight-take-off-data-jfk-airport/M1_final.csv")
print(df)

       MONTH  DAY_OF_MONTH  DAY_OF_WEEK OP_UNIQUE_CARRIER TAIL_NUM DEST  \
0         11             1            5                B6   N828JB  CHS   
1         11             1            5                B6   N992JB  LAX   
2         11             1            5                B6   N959JB  FLL   
3         11             1            5                B6   N999JQ  MCO   
4         11             1            5                DL   N880DN  ATL   
...      ...           ...          ...               ...      ...  ...   
28815      1            31            5                B6   N216JB  ORH   
28816      1            31            5                AA   N104NN  BOS   
28817      1            31            5                AS   N581AS  SEA   
28818      1            31            5                B6   N957JB  SJU   
28819      1            31            5                B6   N958JB  BQN   

       DEP_DELAY  CRS_ELAPSED_TIME  DISTANCE  CRS_DEP_M  ...  Dew Point  \
0             -1        

In [290]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28820 entries, 0 to 28819
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MONTH              28820 non-null  int64  
 1   DAY_OF_MONTH       28820 non-null  int64  
 2   DAY_OF_WEEK        28820 non-null  int64  
 3   OP_UNIQUE_CARRIER  28820 non-null  object 
 4   TAIL_NUM           28820 non-null  object 
 5   DEST               28820 non-null  object 
 6   DEP_DELAY          28820 non-null  int64  
 7   CRS_ELAPSED_TIME   28820 non-null  int64  
 8   DISTANCE           28820 non-null  int64  
 9   CRS_DEP_M          28820 non-null  int64  
 10  DEP_TIME_M         28820 non-null  int64  
 11  CRS_ARR_M          28820 non-null  int64  
 12  Temperature        28820 non-null  int64  
 13  Dew Point          28820 non-null  object 
 14  Humidity           28820 non-null  int64  
 15  Wind               28818 non-null  object 
 16  Wind Speed         288

In [291]:
X = df.drop(['TAXI_OUT'], axis = 1)
Y = df["TAXI_OUT"]
print(X.shape, Y.shape)

(28820, 22) (28820,)


In [292]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

X['OP_UNIQUE_CARRIER'] = LE.fit_transform(X['OP_UNIQUE_CARRIER'].astype(str))
X['TAIL_NUM'] = LE.fit_transform(X['TAIL_NUM'].astype(str))
X['DEST'] = LE.fit_transform(X['DEST'].astype(str))
X['Wind'] = LE.fit_transform(X['Wind'].astype(str))
X['Condition'] = LE.fit_transform(X['Condition'].astype(str))

In [293]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 2)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(25938, 22) (2882, 22) (25938,) (2882,)


In [294]:
rmseL = np.empty(8, dtype = float)

# Label Encoding

## 1. Linear Regression with Label Encoding

In [295]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

LinearReg = LinearRegression()
LinearReg.fit(X_train, Y_train)

pred0 = LinearReg.predict(X_test)
rmseL[0] = mean_squared_error(Y_test, pred0)**0.5
print(rmseL[0])

6.411460932339431


## 2. Ridge Regression with Label Encoding

In [296]:
from sklearn.linear_model import Ridge

RidgeReg = Ridge(alpha = 0.05, normalize = True)
RidgeReg.fit(X_train, Y_train)

pred1 = RidgeReg.predict(X_test)
rmseL[1] = mean_squared_error(Y_test, pred1)**0.5
print(rmseL[1])

6.421772117356294


## 3. Lasso Regression with Label Encoding

In [297]:
from sklearn.linear_model import Lasso

LassoReg = Lasso(alpha = 0.05, normalize = True)
LassoReg.fit(X_train, Y_train)

pred2 = LassoReg.predict(X_test)
rmseL[2] = mean_squared_error(Y_test,pred2)**0.5
print(rmseL[2])

6.705901327790672


## 4. KNN Regression with Label Encoding

In [298]:
from sklearn.neighbors import KNeighborsRegressor

KNNReg = KNeighborsRegressor(n_neighbors = 200)
KNNReg.fit(X_train, Y_train)

pred3 = KNNReg.predict(X_test)
rmseL[3] = mean_squared_error(Y_test,pred3)**0.5
print(rmseL[3])

6.530877671436237


## 5. Support Vector Regression with Label Encoding

In [299]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

SC1 = StandardScaler()
SC2 = StandardScaler()

X1 = SC1.fit_transform(X_train.copy())
X2 = SC2.fit_transform(X_test.copy())

svr = SVR()
svr.fit(X1, Y_train)

pred4 = svr.predict(X2)
rmseL[4] = mean_squared_error(Y_test, pred4)**0.5
print(rmseL[4])

6.138059151798444


## 6. Random Forest with Label Encoding

In [300]:
from sklearn.ensemble import RandomForestRegressor

RandReg = RandomForestRegressor()
RandReg.fit(X_train, Y_train)

pred5 = RandReg.predict(X_test)
rmseL[5] = mean_squared_error(Y_test, pred5)**0.5
print(rmseL[5])

5.509313797100343


## 7. Light GBM model with Label Encoding

In [301]:
from lightgbm import LGBMRegressor

X1 = X_train.drop(["Dew Point"], axis = 1)
X2 = X_test.drop(["Dew Point"], axis = 1)

LGBMReg = LGBMRegressor()
LGBMReg.fit(X1, Y_train)

pred6 = LGBMReg.predict(X2)
rmseL[6] = mean_squared_error(Y_test, pred6)**0.5
print(rmseL[6])

5.6258055863215235


## 8. Naive Bayes (Bayesian Ridge) with Label Encoding

In [302]:
from sklearn.linear_model import BayesianRidge

BayReg = BayesianRidge()
BayReg.fit(X_train, Y_train)

pred7 = BayReg.predict(X_test)
rmseL[7] = mean_squared_error(Y_test, pred7)**0.5
print(rmseL[7])

6.413484980426009


# One Hot Encoding

In [303]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28820 entries, 0 to 28819
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MONTH              28820 non-null  int64  
 1   DAY_OF_MONTH       28820 non-null  int64  
 2   DAY_OF_WEEK        28820 non-null  int64  
 3   OP_UNIQUE_CARRIER  28820 non-null  object 
 4   TAIL_NUM           28820 non-null  object 
 5   DEST               28820 non-null  object 
 6   DEP_DELAY          28820 non-null  int64  
 7   CRS_ELAPSED_TIME   28820 non-null  int64  
 8   DISTANCE           28820 non-null  int64  
 9   CRS_DEP_M          28820 non-null  int64  
 10  DEP_TIME_M         28820 non-null  int64  
 11  CRS_ARR_M          28820 non-null  int64  
 12  Temperature        28820 non-null  int64  
 13  Dew Point          28820 non-null  object 
 14  Humidity           28820 non-null  int64  
 15  Wind               28818 non-null  object 
 16  Wind Speed         288

In [304]:
from sklearn.model_selection import train_test_split

X = df.drop("TAXI_OUT",axis=1)
Y = df["TAXI_OUT"].values
X = pd.get_dummies(X, columns=["MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", "OP_UNIQUE_CARRIER", "TAIL_NUM", "DEST", "Wind", "Condition"])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1)
print(np.shape(X_train), np.shape(X_test), np.shape(Y_train), np.shape(Y_test))

(25938, 2264) (2882, 2264) (25938,) (2882,)


In [305]:
rmseH = np.empty(8, dtype = float)

## 1. Linear Regression with One Hot Encoding

In [306]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

LinearReg = LinearRegression()
LinearReg.fit(X_train, Y_train)

pred0 = LinearReg.predict(X_test)
rmseH[0] = mean_squared_error(Y_test, pred0)**0.5
print(rmseH[0])

88644676.66822751


## 2. Ridge Regression with One Hot Encoding

In [307]:
from sklearn.linear_model import Ridge

RidgeReg = Ridge(alpha = 0.05, normalize = True)
RidgeReg.fit(X_train, Y_train)

pred1 = RidgeReg.predict(X_test)
rmseH[1] = mean_squared_error(Y_test, pred1)**0.5
print(rmseH[1])

6.648756311630615


## 3. Lasso Regression with One Hot Encoding

In [308]:
from sklearn.linear_model import Lasso

LassoReg = Lasso(alpha = 0.05, normalize = True)
LassoReg.fit(X_train, Y_train)

pred2 = LassoReg.predict(X_test)
rmseH[2] = mean_squared_error(Y_test, pred2)**0.5
print(rmseH[2])

6.98378882680972


## 4. KNN regression with One Hot Encoding

In [309]:
from sklearn.neighbors import KNeighborsRegressor

KNNReg = KNeighborsRegressor(n_neighbors = 200)
KNNReg.fit(X_train, Y_train)

pred3 = KNNReg.predict(X_test)
rmseH[3] = mean_squared_error(Y_test, pred3)**0.5
print(rmseH[3])

6.73124694957652


## 5. Support Vector Regression with One Hot Encoding

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

SC1 = StandardScaler()
SC2 = StandardScaler()

X1 = SC1.fit_transform(X_train.copy())
X2 = SC2.fit_transform(X_test.copy())

svr = SVR()

svr.fit(X1, Y_train)

pred4 = svr.predict(X2)
rmseH[4] = mean_squared_error(Y_test, pred4)**0.5
print(rmseH[4])

## 6. Random Forest Regression with One Hot Encoding

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RandReg = RandomForestRegressor()
RandReg.fit(X_train, Y_train)

pred5 = RandReg.predict(X_test)
rmseH[5] = mean_squared_error(Y_test, pred5)**0.5
print(rmseH[5])

## 7. Light GBM model with One Hot Encoding

In [ ]:
from lightgbm import LGBMRegressor

LGBMReg = LGBMRegressor()
LGBMReg.fit(X_train, Y_train)

pred6 = LGBMReg.predict(X_test)
rmseH[6] = mean_squared_error(Y_test, pred6)**0.5
print(rmseH[6])

## 8. Naive Bayes (Bayesian Ridge) with One Hot Encoding

In [ ]:
from sklearn.linear_model import BayesianRidge

BayReg = BayesianRidge()
BayReg.fit(X_train, Y_train)

pred7 = BayReg.predict(X_test)
rmseH[7] = mean_squared_error(Y_test, pred7)**0.5
print(rmseH[7])

In [ ]:
models = ["Linear Regression", "Ridge Regression", "Lasso Regression", "KNN Regression", "Support Vector Regression", "Random forest regression", "Light GBM", "Naive Bayes"]
mod1 = ["LR", "RR", "LSR", "KNNR", "SVR", "RFR", "LGBM", "NBR"]

plt.plot(mod1, rmseL)
plt.plot(mod1, rmseH, 'r')
plt.legend(["Label Encoding", "One Hot Encoding"])
for i in range(8):
    print(mod1[i], ":", models[i])
plt.show()